referenfce: https://github.com/ukairia777/finance_sentiment_corpus

한국어 금융 뉴스 감성분석(긍정, 부정 분류)

In [ ]:
!pip install transformers 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:0

In [ ]:
# 데이터 셋 로드 및 구조 확인 

!wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv

--2023-05-08 05:32:12--  https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319001 (1.3M) [text/plain]
Saving to: ‘finance_data.csv’

finance_data.csv    100%[===================>]   1.26M  --.-KB/s    in 0.03s   

2023-05-08 05:32:12 (42.7 MB/s) - ‘finance_data.csv’ saved [1319001/1319001]



In [ ]:
# 한국어 금융 뉴스 데이터 로드 >> 데이터 프레임에 저장 

import numpy as np 
import pandas as pd 

In [ ]:
df = pd.read_csv('finance_data.csv')

print('샘플의 개수:', len(df))
# 샘플의 개수는 4,846 개 

샘플의 개수: 4846


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   labels        4846 non-null   object
 1   sentence      4846 non-null   object
 2   kor_sentence  4846 non-null   object
dtypes: object(3)
memory usage: 113.7+ KB


In [ ]:
df.head(3)

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."


In [ ]:
df['labels'].value_counts()
# 중립(neural), 긍정(positive), 부정(negative)
# 정수 인코딩 (0,1,2) 변환 필요

neutral     2879
positive    1363
negative     604
Name: labels, dtype: int64

In [ ]:
df['labels'] = df['labels'].replace(['neutral','positive','negative'], [0,1,2])

In [ ]:
df.head(3)

,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."


In [ ]:
df.to_csv('finance_data.csv', index=False, encoding='utf-8')

In [ ]:
# load_dataset 모듈 활용 

from datasets import load_dataset

In [ ]:
all_data = load_dataset(
            "csv",
            data_files={
                "train":"finance_data.csv"
            },
        )

# 현재 train에 모든 데이터가 저장됨 
# load_dataset() 허깅페이스 사에서 만든 transformer library 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bbb2e5ce85ef34d1/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence', 'kor_sentence'],
        num_rows: 4846
    })
})

In [ ]:
# dataset 모듈 활용, train/test set 분류 
# train_test_split()

cs = all_data['train'].train_test_split(0.2)
# 8:2 비율로 훈련 데이터, 검증 데이터 구분, 저장 
train_cs = cs['train']
test_cs = cs['test']

In [ ]:
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3876
})

In [ ]:
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [ ]:
# 검증 데이터 생성하기 
# 훈련용 데이터를 다시 8:2로 구분, 저장 

cs = train_cs.train_test_split(0.2)
train_cs = cs['train']
valid_cs = cs['test']

In [ ]:
# 훈련 데이터 
print(train_cs)
print()
print(valid_cs)
print()
print(test_cs)

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 776
})

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})


In [ ]:
print('두번째 샘플 출력:', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력:', train_cs['labels'][1])

두번째 샘플 출력: `` QPR 8 통합 관리 시스템은 전 세계 기업들이 인간 중심의 비즈니스 프로세스를 제어할 수 있는 가장 빠르고 간편하며 효율적인 방법입니다.
두번째 샘플의 레이블 출력: 1


데이터 셋 전처리

In [ ]:
import numpy as np 
import pandas as pd
import random 
import time
import datetime 
from tqdm import tqdm 

import csv 
import os 

import tensorflow as tf 
import torch 

In [ ]:
# BERT 사용하기 위한 모듈 불러오기 

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
# 패딩(padding) 위한 모듈 불러오기 

from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가지표 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [ ]:
# BERT 구조 
# [CLS] sequence [SEP] 구조

# 훈련 데이터, 검증 데이터, 테스트 데이터를 BERT 구조로 변환 

train_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', train_cs['kor_sentence']))
validation_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', valid_cs['kor_sentence']))
test_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', test_cs['kor_sentence']))

In [ ]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [ ]:
test_sentences[:5]

['[CLS]회사측은 직원 35~45명의 비용 삭감이 목표라고 밝혔다.[SEP]',
 '[CLS]영업이익률은 8.3%로 전년도의 11.8%와 비교된다.[SEP]',
 '[CLS]주식은 나스닥 OMX 헬싱키 규칙에 따라 취득하며, 그 외의 경우에는 회사 자신의 주식 취득과 관련된 규칙에 따라 취득합니다.[SEP]',
 '[CLS]핀란드 보안 및 개인 정보 소프트웨어 솔루션 개발업체 스톤소프트 오이는 스톤게이트 제품의 예비 판매량이 2007년 4분기 3.6mln 유로($5.3mln)로 전년 동기 대비 59pct 증가했다고 2008년 1월 7일 밝혔다.[SEP]',
 '[CLS]CPP의 제조는 뭄바이 인근 코폴리에 있는 바르칠라 공장의 기존 수출 지향 유닛 EOU에서 수행될 것이다.[SEP]']

In [ ]:
# 중립 = 0, 긍정 = 1, 부정 = 2 

test_labels[:5]

[2, 2, 0, 1, 0]

BERT 토큰나이저 이용한 전처리 

- BERT 사용하기 위해서는 tokenizer와 model이 반드시 mapping 되는 관계여야만 함 >> 아래의 이름에 들어가는 모델이름은 반드시 동일해야 함 
- BertTokenizer.from_pretrained('모델이름') 
- BertForSequenceClassification.from_pretrained('모델이름') 

- Tokenizer의 역할은 내부적으로 vocabulary를 갖고 있어서 정수 인코딩을 수행해주는 모듈

In [ ]:
# kor-bert 중 하나니 'klue/bert-base'를 사용 
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
MAX_LEN = 128 
# pre-trained model : 최대 길이 512
# fine-tuning 할 때 데이터 셋 길이 128

def data_to_tensor(sentences, labels): 
  # 정수 인코딩 과정, 각 텍스트를 토큰화한 후에 vocabulary에 mapping 되는 정수 시퀀스로 변환 
  # 예) ['안녕하세요'] >> ['안','녕','하세요'] >> [231,52,45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences: padding 위한 모듈. 주어진 max_len(최대길이)위에 뒤에 0으로 채움 
  # 예) [231,52,45] >> [231,52,45,0,0,0]

  input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
   
  attention_masks =[]

  for seq in input_ids:
     seq_mask = [float(i>0) for i in seq]
     attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks


In [ ]:
# 훈련 데이터, 검증데이터, 테스트 데이터에 대해서 data_to_tensor 함수를 활용
# 정수 인코딩된 데이터, 레이블, 어텐션 마스크를 얻음 

train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [ ]:
print(train_inputs[0])
print()
print(train_masks[0])

tensor([    2,  5125,  2440,    56, 18481, 30062,  6306,  2073,  5125,  2440,
           24,  2429,  4207,  2210,  9893,  2170,    21,  2377,  2015,    12,
           21,    97,    23,  2429,    13,    17,  5125,  2440,    27,  2429,
         3619,  2210,  9893,  2170,    22,  2377,  2015,    12,    24,    97,
           26,  2429,    13,    17,  5125,  2440,  3633,  2429,  4210,  2210,
         9893,  2170,    23,  2377,  2015,    12,    27,    97,    29,  2429,
           13,  5141,  5243,  2138,  4049,  2085,  3834, 28674,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
tokenizer.decode([2])

'[CLS]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

In [ ]:
tokenizer.decode([3678])

'그러나'

In [ ]:
tokenizer.decode([3874])

'생산'

batch_size = 32 
- pytorch의 DataLoader이용, 변환 
- DataLoader: batch 단위로 데이터를 꺼내올 수 있도록 하는 모듈 

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터 크기:', len(train_labels))
print('검증 데이터 크기:', len(validation_labels))
print('테스트 데이터 크기:', len(test_labels))

훈련 데이터 크기: 3100
검증 데이터 크기: 776
테스트 데이터 크기: 970


In [ ]:
# GPU 정상 세팅이 되었는지 확인 

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


모델 로드하기 

- BERT 사용, 텍스트 분류 아키텍처
  - BertForSequenceClassification.from_pretrained('모델명') 
  - label 개수: num_labels

In [ ]:
num_labels = 3

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 최적화(optimizer) 선택 

optimizer = AdamW(model.parameters(),
            lr= 2e-5,
            eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 학습 반복 횟수 설정 (epochs)

epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                num_warmup_steps = 0, 
                                num_training_steps = total_steps)

In [ ]:
def format_time(elapsed): 
   elapsed_rounded = int(round(elapsed))
   return str(datetime.timedelta(seconds=elapsed_rounded)) 
   # hh:mm:ss 

In [ ]:
def metrics(predictions, labels): 
   y_pred = predictions
   y_true = labels

   # 사용 가능한 메트릭들을 사용함 
   accuracy = accuracy_score(y_true, y_pred)
   f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0) 
   f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0) 
   f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
   
   # metric 결과에 대해서도 return 

   metrics = {'accuracy': accuracy, 
              'f1_macro': f1_macro_average,
              'f1_micro': f1_micro_average,
              'f1_weighted': f1_weighted_average
              }

   return metrics

모델 학습 

In [ ]:
# random seed 값 설정 

seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs): 
  print('---------- Epoch {:} / {:} ----------'.format(epoch_i + 1, epochs))
  t0 = time.time()
  total_loss = 0

  model.train() 

  for step, batch in tqdm(enumerate(train_dataloader)): 
    if step % 500 == 0 and not step == 0: 
      elapsed = format_time(time.time() - t0)
      print(' Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)     
    b_input_ids, b_input_mask, b_labels = batch

    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask = b_input_mask,
                    labels = b_labels
                    )  
    loss = outputs[0]     
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
    # threshold 초과시, gradient clipping 해줌 
    optimizer.step()
    scheduler.step()

    model.zero_grad()

  avg_train_loss = total_loss / len(train_dataloader)

  print("")
  print("Average training loss: {0:.4f}".format(avg_train_loss))
  print("Training epoch taken: {:}".format(format_time(time.time() - t0)))



---------- Epoch 1 / 3 ----------


97it [00:59,  1.64it/s]



Average training loss: 0.5814
Training epoch taken: 0:00:59
---------- Epoch 2 / 3 ----------


97it [00:58,  1.65it/s]



Average training loss: 0.3245
Training epoch taken: 0:00:59
---------- Epoch 3 / 3 ----------


97it [01:03,  1.54it/s]


Average training loss: 0.2339
Training epoch taken: 0:01:03


검증 데이터에 대한 평가 

In [ ]:
# eval() : 파이썬에서 문자열로 된 코드 실행 함수 
eval("1+2")

3

In [ ]:
t0 = time.time()
model.eval() 
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader: 
  batch = tuple(t.to(device) for t in batch) 
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad(): 
    outputs = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask = b_input_mask)
    
    # print(outputs[0])

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
      # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 
      # 예를 들어 [3.5134, - 0.308765, -2.11316] >> 0
      accum_logits.append(np.argmax(b))

    for b in label_ids: 
      accum_label_ids.append(b)
  
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))

[0 0 0 0 1 0 0 0 0 0 2 1 0 1 0 0 0 0 1 0 0 1 1 2 0 1 1 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 2 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 2 0 0 2 0 0 0 1 0 1 2 0 2 1
 1 0 0 2 1 2 0 2 0 0 0 0 0 0 1 0 0 0 0 0 2 1 0 0 0 0 0 0 2 0 2 0 0 1 0 1 0
 1 0 0 0 1 2 1 1 1 0 1 0 1 0 0 0 2 2 0 0 0 1 0 0 0 0 2 1 0 0 0 0 2 0 0 1 0
 2 0 0 2 0 2 0 0 2 2 1 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 0 0 1 0 0 2 1 0 0
 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1
 0 0 0 2 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 0 2 1 1 0 0 0 0 0 1 1 0
 2 1 0 1 2 0 1 1 0 0 0 0 2 2 2 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 2 0 0 1
 0 1 0 0 2 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 1 1 0 0 0 0 1 2 0 1 0 0 0
 1 0 0 0 0 1 0 0 0 1 0 0 0 0 2 0 0 0 1 0 0 2 0 1 1 0 0 0 2 0 0 0 0 0 1 0 0
 2 0 2 0 0 1 1 0 0 0 1 0 0 0 0 0 1 2 2 0 2 0 0 0 1 0 0 2 1 0 2 0 0 0 2 0 0
 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 2 2 0 1 1 1 1 0 1 0 0 1 1 0 0 0 2 0 0 1
 0 2 0 0 2 2 0 1 0 2 0 0 1 1 1 0 0 0 1 1 0 0 0 2 1 0 0 1 0 0 0 1 1 2 1 2 0
 1 1 2 0 1 0 0 1 0 1 0 1 

모델 저장과 모델 로드 

In [ ]:
# 폴더 생성 
%mkdir model 

In [ ]:
pwd

'/content'

In [ ]:
path = '/content/model'

In [ ]:
cd /content/model

/content/model


In [ ]:
# 모델 저장 

torch.save(model.state_dict(), "BERT_news_emotion_model.pt")

In [ ]:
# 모델 로드 

model.load_state_dict(torch.load("BERT_news_emotion_model.pt"))

<All keys matched successfully>

테스트 데이터에 대한 평가 

In [ ]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [],[]

for step, batch in tqdm(enumerate(test_dataloader)):
  if step % 100 == 0 and not step == 0: 
    elapsed = format_time(time.time() - t0)
    print('Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask = b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()  
  label_ids = b_labels.to('cpu').numpy()

  for b in logits: 
    accum_logits.append(np.argmax(b))

  for b in label_ids: 
    accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))


31it [00:06,  4.55it/s]

[0 1 0 0 0 0 0 1 0 0 1 1 2 1 0 1 0 0 0 0 2 1 0 1 1 1 0 1 0 0 0 1 1 0 2 1 0
 1 0 1 0 1 0 0 0 2 0 1 2 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 2 1 0 0 0 1 1
 2 0 1 0 0 2 0 0 2 0 1 0 0 2 0 2 0 1 0 2 0 0 0 1 0 2 0 0 1 1 2 0 0 2 0 0 0
 0 2 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 2 0 0 0 0 1 0
 2 0 0 0 1 1 0 0 0 1 1 1 2 1 1 2 1 0 1 1 1 2 0 0 0 1 0 1 0 2 1 0 1 0 1 0 1
 0 0 1 0 0 0 1 0 0 0 1 0 0 0 2 1 1 2 0 0 0 1 0 2 2 0 0 0 0 0 1 1 0 0 0 2 2
 0 1 0 0 0 2 0 0 0 0 1 1 0 2 0 0 1 2 1 0 0 2 0 2 0 0 0 2 0 2 0 2 0 1 1 2 0
 1 0 2 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 2 0 0 0 0 0 0 2 1 0 0 1 0 0 0 0 0 1 2 1 0 1 1 1 0 1 0 2 1 1 1 0 0 0 0 2 2
 0 0 0 2 2 0 2 0 0 0 0 2 0 1 2 1 1 0 2 0 2 0 0 0 0 1 1 0 0 0 2 0 2 2 1 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 2 1 1 0 2 0 1 2 0 2 0 2 1 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1 2 0 1 1 2 1 0 1 0
 0 0 0 0 2 0 0 0 0 0 0 0 1 2 0 0 2 0 0 1 2 0 0 0 0 0 1 2 0 0 1 1 0 2 0 1 1
 0 0 1 1 0 2 0 1 0 0 1 1 

예측 

In [ ]:
from transformers import pipeline
# pipeline : 정형화된 데이터 일 경우 사용 

In [ ]:
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
                return_all_scores=True, function_to_apply='softmax' )
pipe

# return_all_scores=True : 모든 라벨에 대한 확률값 반환 

In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

[[{'label': 'LABEL_0', 'score': 0.02632298693060875}, {'label': 'LABEL_1', 'score': 0.9686517119407654}, {'label': 'LABEL_2', 'score': 0.005025310441851616}]]


In [ ]:
result[0]

[{'label': 'LABEL_0', 'score': 0.02632298693060875},
 {'label': 'LABEL_1', 'score': 0.9686517119407654},
 {'label': 'LABEL_2', 'score': 0.005025310441851616}]

In [ ]:
result[0][0]

{'label': 'LABEL_0', 'score': 0.02632298693060875}

In [ ]:
result[0][0]['label']

'LABEL_0'

In [ ]:
print(result)

[[{'label': 'LABEL_0', 'score': 0.02632298693060875}, {'label': 'LABEL_1', 'score': 0.9686517119407654}, {'label': 'LABEL_2', 'score': 0.005025310441851616}]]


In [ ]:
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
                 function_to_apply='softmax' )

In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

[{'label': 'LABEL_1', 'score': 0.9686517119407654}]


In [ ]:
label_dict = {'LABEL_0': '중립', 'LABEL_1':'긍정', 'LABEL_2':'부정'}

In [ ]:
label_dict['LABEL_0']

'중립'

In [ ]:
def prediction(text): 
   result = pipe(text)

   return [label_dict[result[0]['label']]]

In [ ]:
prediction("매출이 급성장했다")

['긍정']

In [ ]:
prediction("챗지피 등장으로 기존 산업이 위협을 느끼고 있다")

['부정']